# Employment Explorer

CES (Establishment Survey) payrolls and CPS (Household Survey) labor force measures.

In [ ]:
from macro_econ.series import build_ces_tree, build_cps_tree
from macro_econ.clients import FredClient
from macro_econ.transforms import (
    level_change, yoy_change, mom_change, moving_average
)
from macro_econ.viz.charts import line_chart, bar_chart, stacked_bar_contributions, recession_shading
from macro_econ.viz.widgets import build_tree_widget

fred = FredClient()
ces = build_ces_tree()
cps = build_cps_tree()
print('=== CES Hierarchy ===')
print(ces.print_tree()[:1500])
print()
print('=== CPS Hierarchy ===')
print(cps.print_tree())

## Total Nonfarm Payrolls

In [ ]:
nfp = fred.fetch_series('PAYEMS', start_date='2000-01-01')

# Level
fig = line_chart({'Total Nonfarm': nfp}, 'Total Nonfarm Payrolls (Thousands)', yaxis_title='Thousands')
fig = recession_shading(fig)
fig.show()

# Monthly change
nfp_chg = level_change(nfp)
fig = bar_chart({'Monthly Change': nfp_chg.iloc[-48:]}, 'Nonfarm Payrolls Monthly Change', yaxis_title='Thousands')
fig.show()

## Sector Breakdown

In [ ]:
# Fetch supersector-level data
sectors = {
    'Mining/Logging': 'USMINE',
    'Construction': 'USCONS',
    'Manufacturing': 'MANEMP',
    'Trade/Trans/Util': 'USTPU',
    'Information': 'USINFO',
    'Financial': 'USFIRE',
    'Prof/Business': 'USSERV',
    'Edu/Health': 'USEHS',
    'Leisure/Hosp': 'USLAH',
    'Government': 'USGOVT',
}

sector_data = {}
for label, sid in sectors.items():
    sector_data[label] = fred.fetch_series(sid, start_date='2019-01-01')

# Monthly changes by sector (last 12 months)
sector_changes = {label: level_change(df).iloc[-12:] for label, df in sector_data.items()}
fig = stacked_bar_contributions(sector_changes, 'Employment Change by Sector (thousands)', yaxis_title='Thousands')
fig.show()

## Average Hourly Earnings

In [ ]:
ahe = fred.fetch_series('CES0500000003', start_date='2007-01-01')  # AHE total private

fig = line_chart(
    {'AHE Level': ahe, 'AHE YoY %': yoy_change(ahe, 12)},
    'Average Hourly Earnings (Total Private)'
)
fig = recession_shading(fig)
fig.show()

## Household Survey (CPS)

In [ ]:
ur = fred.fetch_series('UNRATE', start_date='2000-01-01')
lfpr = fred.fetch_series('CIVPART', start_date='2000-01-01')
epop = fred.fetch_series('EMRATIO', start_date='2000-01-01')

fig = line_chart(
    {'Unemployment Rate': ur, 'LFPR': lfpr, 'Emp-Pop Ratio': epop},
    'Labor Market Dashboard (%)',
    yaxis_title='Percent'
)
fig = recession_shading(fig)
fig.show()

## Alternative Unemployment Measures (U-1 through U-6)

In [ ]:
u_measures = {
    'U-3 (Official)': 'UNRATE',
    'U-6 (Broadest)': 'U6RATE',
}

u_data = {label: fred.fetch_series(sid, start_date='2000-01-01') for label, sid in u_measures.items()}
fig = line_chart(u_data, 'Alternative Unemployment Measures', yaxis_title='Percent')
fig = recession_shading(fig)
fig.show()

## Demographics

In [ ]:
demo = {
    'Prime Age (25-54)': 'LNS14000060',
    '16-19 years': 'LNS14000012',
    '55+ years': 'LNS14000024',
}

demo_data = {label: fred.fetch_series(sid, start_date='2000-01-01') for label, sid in demo.items()}
fig = line_chart(demo_data, 'Unemployment Rate by Age Group', yaxis_title='Percent')
fig = recession_shading(fig)
fig.show()